### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

alexandrovDataDirectory = os.path.join(getDataDir(), "Alexandrov_LUAD")
alexandrovBaseDataFilePath = os.path.join(alexandrovDataDirectory, "Lung_Adeno_clean_somatic_mutations_for_signature_analysis.txt")

### Parse the damage position data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseAlexandrov import parseAlexandrov

alexandrovMutperiodInputFilePath = parseAlexandrov([alexandrovBaseDataFilePath], hg19FastaFilePath)

### Run the mutperiod pipeline to determine translational and rotational periodicity of the data.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite

hybridNucleosomeMapFilePath = os.path.join(mutperiodHg19Directory, "hg19_hybrid_nucleosome_map", "hg19_hybrid_nucleosome_map.bed")
LCL_NucleosomeMapFilePath = os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_all_mappable", "hg19_LCL_MNase_nuc_map_all_mappable.bed")
NHF1_NucleosomeMapFilePath = os.path.join(mutperiodHg19Directory, "hg19_NHF1_MNase_nuc_map_mappable", "hg19_NHF1_MNase_nuc_map_all_mappable.bed")
nucleosomeMapFilePaths = [hybridNucleosomeMapFilePath, LCL_NucleosomeMapFilePath, NHF1_NucleosomeMapFilePath]
nucleosomeMapNames = [os.path.basename(nucleosomeMapFilePath).rsplit('.', 1)[0] for nucleosomeMapFilePath in nucleosomeMapFilePaths]

runAnalysisSuite(alexandrovMutperiodInputFilePath, nucleosomeMapNames, normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

In [ ]:
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from mutperiodpy.RunNucleosomeMutationAnalysis import runNucleosomeMutationAnalysis
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

runNucleosomeMutationAnalysis(getFilesInDirectory(alexandrovDataDirectory, DataTypeStr.normNucCounts + ".tsv"),
                              outputFilePath = os.path.join(alexandrovDataDirectory, "Alexandrov_LUAD_trinuc_normalized_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)